In [2]:
import os
from openai import OpenAI
import pprint

openai_client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [3]:
from datasets import load_dataset

wine_dataset = load_dataset("alfredodeza/wine-ratings")

Generating train split:   0%|          | 0/32780 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/200 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/200 [00:00<?, ? examples/s]

In [4]:
wine_dataset

DatasetDict({
    train: Dataset({
        features: ['name', 'region', 'variety', 'rating', 'notes'],
        num_rows: 32780
    })
    validation: Dataset({
        features: ['name', 'region', 'variety', 'rating', 'notes'],
        num_rows: 200
    })
    test: Dataset({
        features: ['name', 'region', 'variety', 'rating', 'notes'],
        num_rows: 200
    })
})

In [5]:
def convert_wine_to_string(wine):
    description = f'{wine["name"]} is from {wine["region"]} and is a {wine["variety"]}. {wine["notes"]}'
    
    return description

for wine in list(wine_dataset['test'])[:5]:
    print(convert_wine_to_string(wine))
    print()

Jim Barry Lodge Hill Shiraz 2008 is from Clare Valley, South Australia, Australia and is a Red Wine. This wine has deep, dark red/black hues, lifted aromas of mulberry, raspberry, coffee and kirsch. The bramble, chocolate, methol and rosemary fl avours that fi ll the mouth are present in abundance, with delicate, sensual and ripe tannins playing a structural role. This wine is ideally suited to good food, good friends and good conversation.

Argyle Reserve Pinot Noir 2005 is from Willamette Valley, Oregon and is a Red Wine. This Pinot Noir has a seductively rich, super dark ruby-violet color. Aromas leap out of the glass with black cherry, black cherry leather and cedar, along with a ripping portion of cinnamon spice. The wine finishes with a hint of violet flower patch. Use of new oak was delicate, so the aromas are authentic Pinot Noir.  

Cambria Katherine's Vineyard Chardonnay 1997 is from Central Coast, California and is a White Wine. The 1997 Katherines Vineyard Chardonnay is a r

In [6]:
import random

wines = random.sample(list(wine_dataset['test']), 5)

In [7]:
def pretty_print(data, indent=0):
    pp = pprint.PrettyPrinter(indent=indent)
    for item in data:
        for key, value in item.items():
            print(f"{key}: {value}")
        print()
    print()


In [8]:
# Define a dictionary for prices

prices = {  # https://openai.com/pricing
    'gpt-3.5-turbo': {
        'input': 0.0005 / 1000,  # per token
        'output': 0.0015 / 1000  # per token
    },
    'gpt-4': {
        'input': 0.03 / 1000,  # per token
        'output': 0.06 / 1000  # per token
    }
}


def generate_openai_response(prompt, model='gpt-4', **kwargs):
    pretty_print(prompt)
    chat_completion = openai_client.chat.completions.create(
        messages=prompt,
        model=model,
        **kwargs
    )

    # Calculate the cost
    input_tokens = chat_completion.usage.prompt_tokens
    output_tokens = chat_completion.usage.completion_tokens
    input_cost = input_tokens * prices[model]['input']
    output_cost = output_tokens * prices[model]['output']
    total_cost = input_cost + output_cost
    return chat_completion, {'input_cost': input_cost, 'output_cost': output_cost, 'total_cost': total_cost}

In [9]:
user_description = 'This client hates red wines'

In [10]:
newline = '\n---\n'
res, price = generate_openai_response(
[
    {'role': 'system', 'content': 'You are a wine bot that helps clients understand what kind of wine they want. Given a list of wines and a description of the client. Tell me what wines they want by giving me the only the names of the wines. Include a reason preceding each pick to explain to the user why they might like it.'}, 
    {'role': 'user', 'content': f'User: {user_description}\nWines:\n{f"{newline}".join([convert_wine_to_string(w) for w in wines])}'}
])
    
print(res.choices[0].message.content)

role: system
content: You are a wine bot that helps clients understand what kind of wine they want. Given a list of wines and a description of the client. Tell me what wines they want by giving me the only the names of the wines. Include a reason preceding each pick to explain to the user why they might like it.

role: user
content: User: This client hates red wines
Wines:
Argiolas S'elegas 2005 is from Sardinia, Italy and is a White Wine. Pale yellow color, with a fruity bouquet. Full bodied and rich with a lingering aftertaste that is characteristic of Nuragus. Made from 100% Nuragus. 
---
La Mozza Maremma Toscana Aragone 2010 is from Tuscany, Italy and is a Red Wine. Intense ruby red with reflections of violet. The Sangiovese and Alicante contribute to the structure and complexity of the blend with spice and licorice flavors. The Syrah and Carignan contribute a fresh dark berry component. The wine hints at a Southern Rhone style with black olive aromas, and ripe fruit.
---
Chateau L

In [11]:
price

{'input_cost': 0.015569999999999999,
 'output_cost': 0.00492,
 'total_cost': 0.020489999999999998}

In [14]:
res, price = generate_openai_response(
[
    {'role': 'system', 'content': 'You are a wine bot that helps clients understand what kind of wine they want. Given a list of wines and a description of the client. Tell me what wines they want by giving me the only the names of the wines. Include a reason preceding each pick to explain to the user why they might like it.'}, 
    {'role': 'user', 'content': f'User: {user_description}\nWines:\n{f"{newline}".join([convert_wine_to_string(w) for w in wines])}'}
],
model='gpt-3.5-turbo'
)

role: system
content: You are a wine bot that helps clients understand what kind of wine they want. Given a list of wines and a description of the client. Tell me what wines they want by giving me the only the names of the wines. Include a reason preceding each pick to explain to the user why they might like it.

role: user
content: User: This client hates red wines
Wines:
Argiolas S'elegas 2005 is from Sardinia, Italy and is a White Wine. Pale yellow color, with a fruity bouquet. Full bodied and rich with a lingering aftertaste that is characteristic of Nuragus. Made from 100% Nuragus. 
---
La Mozza Maremma Toscana Aragone 2010 is from Tuscany, Italy and is a Red Wine. Intense ruby red with reflections of violet. The Sangiovese and Alicante contribute to the structure and complexity of the blend with spice and licorice flavors. The Syrah and Carignan contribute a fresh dark berry component. The wine hints at a Southern Rhone style with black olive aromas, and ripe fruit.
---
Chateau L

In [16]:
price

{'input_cost': 0.00025949999999999997,
 'output_cost': 0.00016800000000000002,
 'total_cost': 0.0004275}